In [2]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train['Cabin'].fillna('Z/9999/Z', inplace=True)
test['Cabin'].fillna('Z/9999/Z', inplace=True)

train['Cabin_deck'] = train['Cabin'].apply(lambda x: x.split('/')[0])
train['Cabin_number'] = train['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
train['Cabin_side'] = train['Cabin'].apply(lambda x: x.split('/')[2])
test['Cabin_deck'] = test['Cabin'].apply(lambda x: x.split('/')[0])
test['Cabin_number'] = test['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
test['Cabin_side'] = test['Cabin'].apply(lambda x: x.split('/')[2])

train.loc[train['Cabin_deck']=='Z', 'Cabin_deck']=np.nan
train.loc[train['Cabin_number']==9999, 'Cabin_number']=np.nan
train.loc[train['Cabin_side']=='Z', 'Cabin_side']=np.nan
test.loc[test['Cabin_deck']=='Z', 'Cabin_deck']=np.nan
test.loc[test['Cabin_number']==9999, 'Cabin_number']=np.nan
test.loc[test['Cabin_side']=='Z', 'Cabin_side']=np.nan

train = train.drop('Cabin', axis = 1)
test = test.drop('Cabin', axis = 1)

train['group'] = train['PassengerId'].apply(lambda x: x.split('_')[0]).astype(int)
test['group'] = test['PassengerId'].apply(lambda x: x.split('_')[0]).astype(int)

train['group_size'] = train['group'].apply(lambda x: train['group'].value_counts()[x])
test['group_size'] = test['group'].apply(lambda x: test['group'].value_counts()[x])

train_g_cd = train.groupby('group')['Cabin_deck'].value_counts()
train_g_cd = pd.DataFrame(train_g_cd)
train_g_cd.columns = ['count']
train_g_cd = train_g_cd.reset_index()
train_g_cd = train_g_cd.drop_duplicates('group')
test_g_cd = test.groupby('group')['Cabin_deck'].value_counts()
test_g_cd = pd.DataFrame(test_g_cd)
test_g_cd.columns = ['count']
test_g_cd = test_g_cd.reset_index()
test_g_cd = test_g_cd.drop_duplicates('group')

train_g_cn = train.groupby('group')['Cabin_number'].value_counts()
train_g_cn = pd.DataFrame(train_g_cn)
train_g_cn.columns = ['count']
train_g_cn = train_g_cn.reset_index()
train_g_cn = train_g_cn.drop_duplicates('group')
test_g_cn = test.groupby('group')['Cabin_number'].value_counts()
test_g_cn = pd.DataFrame(test_g_cn)
test_g_cn.columns = ['count']
test_g_cn = test_g_cn.reset_index()
test_g_cn = test_g_cn.drop_duplicates('group')

train_g_cs = train.groupby('group')['Cabin_side'].value_counts()
train_g_cs = pd.DataFrame(train_g_cs)
train_g_cs.columns = ['count']
train_g_cs = train_g_cs.reset_index()
train_g_cs = train_g_cs.drop_duplicates('group')
test_g_cs = test.groupby('group')['Cabin_side'].value_counts()
test_g_cs = pd.DataFrame(test_g_cs)
test_g_cs.columns = ['count']
test_g_cs = test_g_cs.reset_index()
test_g_cs = test_g_cs.drop_duplicates('group')

cab_null_idx = train[train['Cabin_deck'].isnull()].index
test_cab_null_idx = test[test['Cabin_deck'].isnull()].index

train.loc[cab_null_idx, 'Cabin_deck'] = train.loc[cab_null_idx, 'group'].apply(lambda x: train_g_cd['Cabin_deck'].mode()[0] if sum(train_g_cd['group']==x)==0 else train_g_cd[train_g_cd['group']==x]['Cabin_deck'].values[0])
train.loc[cab_null_idx, 'Cabin_number'] = train.loc[cab_null_idx, 'group'].apply(lambda x: train_g_cn['Cabin_number'].mode()[0] if sum(train_g_cn['group']==x)==0 else train_g_cn[train_g_cn['group']==x]['Cabin_number'].values[0])
train.loc[cab_null_idx, 'Cabin_side'] = train.loc[cab_null_idx, 'group'].apply(lambda x: train_g_cs['Cabin_side'].mode()[0] if sum(train_g_cs['group']==x)==0 else train_g_cs[train_g_cs['group']==x]['Cabin_side'].values[0])
test.loc[test_cab_null_idx, 'Cabin_deck'] = test.loc[test_cab_null_idx, 'group'].apply(lambda x: test_g_cd['Cabin_deck'].mode()[0] if sum(test_g_cd['group']==x)==0 else test_g_cd[test_g_cd['group']==x]['Cabin_deck'].values[0])
test.loc[test_cab_null_idx, 'Cabin_number'] = test.loc[test_cab_null_idx, 'group'].apply(lambda x: test_g_cn['Cabin_number'].mode()[0] if sum(test_g_cn['group']==x)==0 else test_g_cn[test_g_cn['group']==x]['Cabin_number'].values[0])
test.loc[test_cab_null_idx, 'Cabin_side'] = test.loc[test_cab_null_idx, 'group'].apply(lambda x: test_g_cs['Cabin_side'].mode()[0] if sum(test_g_cs['group']==x)==0 else test_g_cs[test_g_cs['group']==x]['Cabin_side'].values[0])


train['Cabin_number'] = train['Cabin_number'].astype(int)
test['Cabin_number'] = test['Cabin_number'].astype(int)

train['Destination'].fillna(train['Destination'].mode()[0], inplace = True)
test['Destination'].fillna(test['Destination'].mode()[0], inplace = True)

train.drop(['group', 'group_size'], axis = 1, inplace = True)
test.drop(['group', 'group_size'], axis = 1, inplace = True)